In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import re

# CGV

## Event List

In [2]:
def get_CGV_GoodsIdx(event_idx):
    url = "https://m.cgv.co.kr/Event/GiveawayEventDetail.aspx/GetGiveawayEventDetail"
    body = {
        "eventIndex": event_idx,
        "giveawayIndex": None
    }
    response = requests.post(url, json=body)
    text = response.text
    loads = json.loads(text)
    goods_idx = loads["d"]["GiveawayItemList"][0]["GiveawayItemCode"]
    return goods_idx

def get_CGV_EventList(movie_filter='', goods_filter=''):
    url = "https://m.cgv.co.kr/Event/GiveawayEventList.aspx/GetGiveawayEventList"
    body = {
        "theaterCode": "",
        "pageNumber": "1",
        "pageSize": "100"
    }
    response = requests.post(url, json=body)
    text = response.text
    loads = json.loads(text)
    soup = BeautifulSoup(loads['d']['List'])
    event_list = soup.find_all("li")
    event_df = pd.DataFrame()
    p = re.compile(r'\[([\d\D]+)\]') # 영화이름 추출 정규식
    for event in event_list:
        event_idx = event['data-eventidx']
        goods_idx = get_CGV_GoodsIdx(event_idx)
        event_name = event.find("strong").get_text()
        event_period = event.find("span").get_text()
        m = p.match(event_name)
        movie_name = m.group()[1:-1]
        goods_name = event_name[m.end():] # 띄어쓰기 했다안했다
        event_dict = {"event_idx":event_idx, "goods_idx":goods_idx, "movie":movie_name, "goods":goods_name, "period":event_period}
        event_df = pd.concat([event_df, pd.DataFrame.from_dict(event_dict, orient='index').T], ignore_index=True)
    event_df = event_df[
        (event_df["movie"].str.contains(movie_filter)) &
        (event_df["goods"].str.contains(goods_filter))
    ]
    # 날짜 필터도 추가 예정
    # 위에서 필터링
    return event_df

In [3]:
event_df = get_CGV_EventList(goods_filter="TTT")
event_df

,event_idx,goods_idx,movie,goods,period
0,202403253872,2024032538723859,고질라 X 콩-뉴 엠파이어,TTT,2024.03.27 ~ 2024.04.26
5,202403183820,2024031838203819,극장판 스파이 패밀리 코드-화이트,TTT,2024.03.20 ~ 2024.04.19
6,202404013900,2024040139003887,기동전사 건담 시드 프리덤,TTT,2024.04.03 ~ 2024.05.02
13,202404093946,2024040939463943,슈가│어거스트 디 투어~,TTT 증정,2024.04.10 ~ 2024.04.16
23,202404013901,2024040139013888,유미의 세포들 더 무비,TTT,2024.04.03 ~ 2024.05.02
26,202404083925,2024040839253922,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
30,202403043767,2024030437673750,패스트 라이브즈,TTT,2024.03.06 ~ 2024.04.21


## Goods

- type2: 재고보유가능성높음
- type3: 재고소진임박
- type4: 마감

In [4]:
def get_CGV_Theaters(giveawayIndex):
    url = "https://m.cgv.co.kr/Event/GiveawayEventDetail.aspx/GetGiveawayTheaterInfo"
    body = {
        "giveawayIndex": giveawayIndex,
        "areaCode": None
    }
    response = requests.post(url, json=body)
    text = response.text
    loads = json.loads(text)
    AreaList = loads["d"]["AreaList"]
    Theaters = pd.DataFrame()
    for Area in AreaList:
        AreaCode = Area["AreaCode"]
        body = {
            "giveawayIndex": giveawayIndex,
             "areaCode": AreaCode
        }
        response = requests.post(url, json=body)
        text = response.text
        loads = json.loads(text)
        Theater = pd.DataFrame(loads["d"]["TheaterList"])
        Theaters = pd.concat([Theaters, Theater[["TheaterName", "TheaterCode", "CountTypeCode"]]], ignore_index=True)
    return Theaters

In [5]:
giveawayIndex = '2024032538723859'
Theaters = get_CGV_Theaters(giveawayIndex)
Theaters

,TheaterName,TheaterCode,CountTypeCode
0,CGV 강남,0056,type2
1,CGV 강변,0001,type4
2,CGV 구로,0010,type2
3,CGV 동대문,0252,type2
4,CGV 등촌,0230,type2
...,...,...,...
59,CGV 서전주,0225,type2
60,CGV 전주고사,0213,type4
61,CGV 목포평화광장,0280,type2
62,CGV 순천신대,0268,type4


In [8]:
def Get_CGV_GoodsStatus(movie_filter="", goods_filter=""):
    event_df = get_CGV_EventList(movie_filter=movie_filter, goods_filter=goods_filter)
    GoodsStatus = pd.DataFrame()
    for idx in event_df.index:
        giveawayIndex = event_df.loc[idx]["goods_idx"]
        Theaters = get_CGV_Theaters(giveawayIndex)
        Theaters["movie"] =  event_df.loc[idx]["movie"]
        Theaters["goods"] =  event_df.loc[idx]["goods"]
        Theaters["period"] =  event_df.loc[idx]["period"]
        GoodsStatus = pd.concat([GoodsStatus, Theaters], ignore_index=True)
    return GoodsStatus

In [9]:
GoodsStatus = Get_CGV_GoodsStatus(movie_filter="쿵푸팬더", goods_filter="TTT")
GoodsStatus

,TheaterName,TheaterCode,CountTypeCode,movie,goods,period
0,CGV 강남,0056,type3,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
1,CGV 강변,0001,type4,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
2,CGV 구로,0010,type4,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
3,CGV 동대문,0252,type2,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
4,CGV 등촌,0230,type4,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
...,...,...,...,...,...,...
61,CGV 서전주,0225,type2,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
62,CGV 전주고사,0213,type3,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
63,CGV 목포평화광장,0280,type3,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
64,CGV 순천신대,0268,type4,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09


# LotteCinema

In [2]:
def get_LC_EventList():
    url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    # 특별기획전 "EventClassificationCode":"10"
    body = {
        "ParamList": '{"MethodName":"GetEventLists","channelType":"","osType":"","osVersion":"","EventClassificationCode":"20","SearchText":"","CinemaID":"","PageNo":1,"PageSize":100,"MemberNo":"0"}'
    }
    response = requests.post(url, data=body)
    text = response.text
    loads = json.loads(text)
    event_df = pd.DataFrame(loads["Items"])

    goods_info = [get_LC_GoodsInfo(x) for x in event_df['EventID']]
    event_df = pd.concat([event_df[["EventID", "EventName", "ProgressStartDate", "ProgressEndDate"]], 
                           pd.json_normalize(goods_info)[["FrGiftID", "FrGiftNm"]]],
                         axis=1)

    event_df = event_df[~event_df['FrGiftID'].isna()].reset_index(drop=True)

    p = re.compile(r'\<([\d\D]+)\>') # 영화이름 추출 정규식
    event_df["movie_name"] = [
        '' if p.match(x) is None else p.match(x).group()[1:-1] 
        for x in event_df['EventName']
    ]

    p = re.compile(r'\,([\d\D]+)\)') # 굿즈이름 추출 정규식
    event_df["goods_name"] = [
        '' if len(p.findall(x))==0 else p.findall(x)[0].replace(' ', '')
        for x in event_df['FrGiftNm']
    ]
    return event_df

def get_LC_GoodsInfo(event_idx):
    # EventID, FrGiftID, FrGiftNm
    url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    body = {
        "ParamList": '{"MethodName":"GetInfomationDeliveryEventDetail","channelType":"","osType":"","osVersion":"","EventID":%s}' % (event_idx)
    }
    response = requests.post(url, data=body)
    text = response.text
    loads = json.loads(text)
    goods_info = loads["InfomationDeliveryEventDetail"][0]["GoodsGiftItems"]
    if len(goods_info) == 0: 
        goods_info = None
    else:
        goods_info = goods_info[0]
    return goods_info

In [3]:
event_df = get_LC_EventList()
event_df

,EventID,EventName,ProgressStartDate,ProgressEndDate,FrGiftID,FrGiftNm,movie_name,goods_name
0,201010016924197,<남은 인생 10년>3주차 주말 현장증정이벤트,2024.04.20,2024.04.26,9190,"<남은 인생 10년>3주차 주말 굿즈(4/20토~4/26금,엽서세트)",남은 인생 10년,엽서세트
1,201010016924190,<꽃다발 같은 사랑을 했다>2주차 현장증정이벤트,2024.04.17,2024.04.23,9168,"<꽃다발~>2주차 굿즈(4/17수~4/23화,A3포스터)",꽃다발 같은 사랑을 했다,A3포스터
2,201010016924191,<악은 존재하지 않는다>4주차 현장증정이벤트,2024.04.17,2024.04.23,9169,"<악은 존재하지 않는다>4주차 굿즈(4/17수~4/23화,스페셜포스터)",악은 존재하지 않는다,스페셜포스터
3,201010016924192,<땅에 쓰는 시>1주차 현장 증정 이벤트,2024.04.17,2024.04.23,9170,"<땅에 쓰는 시>1주차 굿즈(4/17수~4/23화,스페셜 포스터)",땅에 쓰는 시,스페셜포스터
4,201010016924193,<기생수 파트2>1주차 현장증정 이벤트,2024.04.17,2024.04.23,9171,"<기생수 파트2>1주차 굿즈(4/17수~4/23화, 적박포스터)",기생수 파트2,적박포스터
5,201010016924194,<유미의 세포들>3주차 현장증정이벤트,2024.04.17,2024.04.23,9172,"<유미의 세포들>3주차 굿즈(4/17수~4/23화, 엽서4종세트)",유미의 세포들,엽서4종세트
6,201010016924195,<남은 인생10년> 3주차 현장증정이벤트,2024.04.17,2024.04.23,9175,"<남은 인생10년> 3주차 굿즈(4/17수~4/23화, 포스터2종세트)",남은 인생10년,포스터2종세트
7,201010016924189,이승윤콘서트 3주차 현장이벤트,2024.04.05,2024.04.26,9163,"<이승윤>3주차 굿즈(4/5 금~상영종료시,가사티켓)",,가사티켓


In [5]:
def get_LC_Theaters(EventID, FrGiftID):
    url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    body = {
        "ParamList": '{"MethodName":"GetCinemaGoods","channelType":"","osType":"","osVersion":"","EventID":%s, "GiftID":%s}' % (EventID, FrGiftID)
    }
    response = requests.post(url, data=body)
    text = response.text
    loads = json.loads(text)
    Theaters = pd.DataFrame(loads["CinemaDivisionGoods"])
    return Theaters

In [12]:
EventID = "201010016924197"
FrGiftID = "9190"
Theaters = get_LC_Theaters(EventID, FrGiftID)
Theaters

,DivisionCode,DetailDivisionCode,CinemaID,CinemaNameKR,CinemaNameUS,SortSequence,Cnt,DetailDivisionNameKR,DetailDivisionNameUS
0,1,0001,1004,건대입구,Konkuk Univ,6,50,서울,Seoul
1,1,0001,1009,김포공항,Gimpo Int'l Airport,7,50,서울,Seoul
2,1,0001,1003,노원,Nowon,8,100,서울,Seoul
3,1,0001,1007,신림,Sillim,18,100,서울,Seoul
4,1,0001,1002,영등포,Yeongdeungpo,20,100,서울,Seoul
5,1,0001,1016,월드타워,WorldTower,22,100,서울,Seoul
6,1,0001,1008,청량리,Cheongnyangni,25,50,서울,Seoul
7,1,0002,3008,부평역사,Bupyeong Sation,16,100,경기/인천,Gyeonggi/Incheon
8,1,0002,3018,평촌(범계역),Pyeongchon,46,100,경기/인천,Gyeonggi/Incheon
9,1,0004,6004,수완(아울렛),Suwan,71,100,전라/광주,Cholla/Gwangju


In [13]:
def Get_LC_GoodsStatus(movie_filter="", goods_filter=""):
    event_df = get_LC_EventList()
    event_df = event_df[
        (event_df["EventName"].str.contains(movie_filter)) &
        (event_df["FrGiftNm"].str.contains(goods_filter))
    ]
    GoodsStatus = pd.DataFrame()
    for idx in event_df.index:
        EventID = event_df.loc[idx]["EventID"]
        FrGiftID = event_df.loc[idx]["FrGiftID"]
        Theaters = get_LC_Theaters(EventID, FrGiftID)
        Theaters["movie"] =  event_df.loc[idx]["movie_name"]
        Theaters["goods"] =  event_df.loc[idx]["goods_name"]
        Theaters["period"] =  event_df.loc[idx]["ProgressStartDate"] + '~' + event_df.loc[idx]["ProgressEndDate"]
        GoodsStatus = pd.concat([GoodsStatus, Theaters], ignore_index=True)
    return GoodsStatus

In [14]:
Get_LC_GoodsStatus(goods_filter='엽서')

,DivisionCode,DetailDivisionCode,CinemaID,CinemaNameKR,CinemaNameUS,SortSequence,Cnt,DetailDivisionNameKR,DetailDivisionNameUS,movie,goods,period
0,1,0001,1004,건대입구,Konkuk Univ,6,50,서울,Seoul,남은 인생 10년,엽서세트,2024.04.20~2024.04.26
1,1,0001,1009,김포공항,Gimpo Int'l Airport,7,50,서울,Seoul,남은 인생 10년,엽서세트,2024.04.20~2024.04.26
2,1,0001,1003,노원,Nowon,8,100,서울,Seoul,남은 인생 10년,엽서세트,2024.04.20~2024.04.26
3,1,0001,1007,신림,Sillim,18,100,서울,Seoul,남은 인생 10년,엽서세트,2024.04.20~2024.04.26
4,1,0001,1002,영등포,Yeongdeungpo,20,100,서울,Seoul,남은 인생 10년,엽서세트,2024.04.20~2024.04.26
5,1,0001,1016,월드타워,WorldTower,22,100,서울,Seoul,남은 인생 10년,엽서세트,2024.04.20~2024.04.26
6,1,0001,1008,청량리,Cheongnyangni,25,50,서울,Seoul,남은 인생 10년,엽서세트,2024.04.20~2024.04.26
7,1,0002,3008,부평역사,Bupyeong Sation,16,100,경기/인천,Gyeonggi/Incheon,남은 인생 10년,엽서세트,2024.04.20~2024.04.26
8,1,0002,3018,평촌(범계역),Pyeongchon,46,100,경기/인천,Gyeonggi/Incheon,남은 인생 10년,엽서세트,2024.04.20~2024.04.26
9,1,0004,6004,수완(아울렛),Suwan,71,100,전라/광주,Cholla/Gwangju,남은 인생 10년,엽서세트,2024.04.20~2024.04.26


#### 번외 수원 기획전 

In [7]:
EventID = "101010014224011"
url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
body = {
    "ParamList": '{"MethodName":"GetInfomationDeliveryEventDetail","channelType":"","osType":"","osVersion":"","EventID":%s}' % (EventID)
}
response = requests.post(url, data=body)
text = response.text
loads = json.loads(text)
suwon_list = loads["InfomationDeliveryEventDetail"][0]["GoodsGiftItems"]

suwon_list2 = []
for suwon in suwon_list:
    EventID = suwon["EventID"]
    FrGiftID = suwon["FrGiftID"]
    url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    body = {
        "ParamList": '{"MethodName":"GetCinemaGoods","channelType":"","osType":"","osVersion":"","EventID":%s, "GiftID":%s}' % (EventID, FrGiftID)
    }
    response = requests.post(url, data=body)
    text = response.text
    loads = json.loads(text)
    suwon.update(loads["CinemaDivisionGoods"][0])
    suwon_list2.append(suwon)
    #     print(loads["CinemaDivisionGoods"])
pd.DataFrame(suwon_list2)

,EventID,FrGiftID,FrGiftNm,DivisionCode,DetailDivisionCode,CinemaID,CinemaNameKR,CinemaNameUS,SortSequence,Cnt,DetailDivisionNameKR,DetailDivisionNameUS
0,101010014224011,9183,수원 특별 기획전(스파이더맨 어크로스 더 유니버스),1,0002,3024,수원(수원역),Suwon,23,150,경기/인천,Gyeonggi/Incheon
1,101010014224011,9184,수원 특별 기획전(스파이더맨 노웨이 홈 FUN),1,0002,3024,수원(수원역),Suwon,23,200,경기/인천,Gyeonggi/Incheon
2,101010014224011,9185,수원 특별 기획전(그란투리스모),1,0002,3024,수원(수원역),Suwon,23,400,경기/인천,Gyeonggi/Incheon
3,101010014224011,9186,수원 특별 기획전(탑건: 매버릭),1,0002,3024,수원(수원역),Suwon,23,1050,경기/인천,Gyeonggi/Incheon
4,101010014224011,9187,수원 특별 기획전(언어의 정원),1,0002,3024,수원(수원역),Suwon,23,1050,경기/인천,Gyeonggi/Incheon
5,101010014224011,9188,수원 특별 기획전(날씨의 아이),1,0002,3024,수원(수원역),Suwon,23,900,경기/인천,Gyeonggi/Incheon
